In [83]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [84]:
season = 2017
fifa_version = (season%2000) + 1
base_url = 'https://www.transfermarkt.us/premier-league/torschuetzenliste/wettbewerb/GB1/plus/?saison_id={}'
url = base_url.format(season)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
}

In [85]:
page = requests.get(url, headers=headers)


In [86]:
soup = BeautifulSoup(page.text)

In [87]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<script data-description="sourcepoint stub code" type="text/javascript">
    !function () { var e = function () { var e, t = "__tcfapiLocator", a = [], n = window; for (; n;) { try { if (n.frames[t]) { e = n; break } } catch (e) { } if (n === window.top) break; n = n.parent } e || (!function e() { var a = n.document, r = !!n.frames[t]; if (!r) if (a.body) { var i = a.createElement("iframe"); i.style.cssText = "display:none", i.name = t, a.body.appendChild(i) } else setTimeout(e, 5); return !r }(), n.__tcfapi = function () { for (var e, t = arguments.length, n = new Array(t), r = 0; r < t; r++)n[r] = arguments[r]; if (!n.length) return a; if ("setGdprApplies" === n[0]) n.length > 3 && 2 === parseInt(n[1], 10) && "boolean" == typeof n[3] && (e = n[3], "function" == typeof n[2] && n[2]("set", !0)); else if ("ping" === n[0]) { var i = { gdprApplies: e, cmpLoaded: !1, cmpStatus: "stub" }; "function" == typeof n[2] && n[2](i) } else a.push(n) }, n.add

In [88]:
table = soup.find_all('td', class_='zentriert hauptlink')
table

[<td class="zentriert hauptlink"><a href="/mohamed-salah/leistungsdaten/spieler/148455/saison/2017/wettbewerb/GB1" title="Mohamed Salah">32</a></td>,
 <td class="zentriert hauptlink"><a href="/harry-kane/leistungsdaten/spieler/132098/saison/2017/wettbewerb/GB1" title="Harry Kane">30</a></td>,
 <td class="zentriert hauptlink"><a href="/sergio-aguero/leistungsdaten/spieler/26399/saison/2017/wettbewerb/GB1" title="Sergio Agüero">21</a></td>,
 <td class="zentriert hauptlink"><a href="/jamie-vardy/leistungsdaten/spieler/197838/saison/2017/wettbewerb/GB1" title="Jamie Vardy">20</a></td>,
 <td class="zentriert hauptlink"><a href="/raheem-sterling/leistungsdaten/spieler/134425/saison/2017/wettbewerb/GB1" title="Raheem Sterling">18</a></td>,
 <td class="zentriert hauptlink"><a href="/romelu-lukaku/leistungsdaten/spieler/96341/saison/2017/wettbewerb/GB1" title="Romelu Lukaku">16</a></td>,
 <td class="zentriert hauptlink"><a href="/roberto-firmino/leistungsdaten/spieler/131789/saison/2017/wettbew

In [89]:
# <td class="zentriert">1</td><td> <table class="inline-table">

In [90]:
type(table[0])

bs4.element.Tag

In [91]:
df = pd.DataFrame(columns=["name", "goals", "season"])
for td in table:
    a_tag = td.find('a')  # find the <a> tag inside the <td>
    if a_tag:
        player_name = a_tag.get('title')     # "Alexis Sánchez"
        goals = a_tag.text.strip()           # "9" (as string)
        goals = int(goals) if goals.isdigit() else None  # convert safely
        df.loc[len(df)] = [player_name, goals, fifa_version]
df

,name,goals,season
0,Mohamed Salah,32,18
1,Harry Kane,30,18
2,Sergio Agüero,21,18
3,Jamie Vardy,20,18
4,Raheem Sterling,18,18
5,Romelu Lukaku,16,18
6,Roberto Firmino,15,18
7,Alexandre Lacazette,14,18
8,Gabriel Jesus,13,18
9,Riyad Mahrez,12,18


In [92]:
paginations = soup.find_all('li', class_="tm-pagination__list-item")
paginations

[<li class="tm-pagination__list-item tm-pagination__list-item--active"><a class="tm-pagination__link" href="/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus/" title="Page 1">1</a></li>,
 <li class="tm-pagination__list-item"><a class="tm-pagination__link" href="/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/2" title="Page 2">2</a></li>,
 <li class="tm-pagination__list-item"><a class="tm-pagination__link" href="/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/3" title="Page 3">3</a></li>,
 <li class="tm-pagination__list-item"><a class="tm-pagination__link" href="/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/4" title="Page 4">4</a></li>,
 <li class="tm-pagination__list-item"><a class="tm-pagination__link" href="/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/5" title="Page 5">5</a></li>,
 <li class="tm-pagination__list-item"><a class="tm-pagination__link" href=

In [93]:
other_pages =[]
for page in paginations[1:]:
    a_tag = page.find('a')
    if a_tag:
        link = "https://www.transfermarkt.us" + a_tag.get('href')
        other_pages.append(link)
other_pages # List of the rest of the goal scorers from the season

['https://www.transfermarkt.us/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/2',
 'https://www.transfermarkt.us/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/3',
 'https://www.transfermarkt.us/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/4',
 'https://www.transfermarkt.us/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/5',
 'https://www.transfermarkt.us/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/6',
 'https://www.transfermarkt.us/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/7',
 'https://www.transfermarkt.us/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/8',
 'https://www.transfermarkt.us/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/9',
 'https://www.transfermarkt.us/premier-league/torschuetzenliste/wettbewerb/GB1/saison_id/2017/plus//page/10',
 'https://www.tran

In [94]:
# It looks like there is some repeats pages (2 and 10) so we will have to clean that up in the dataframe

In [95]:
for page_url in other_pages:
    page = requests.get(page_url, headers=headers)
    soup = BeautifulSoup(page.text)
    new_table = soup.find_all('td', class_='zentriert hauptlink')
    for td in new_table:
        a_tag = td.find('a')  # find the <a> tag inside the <td>
        if a_tag:
            player_name = a_tag.get('title')     # "Alexis Sánchez"
            if (df['name'] == player_name).any():
                continue
            else:
                goals = a_tag.text.strip()           # "9" (as string)
                goals = int(goals) if goals.isdigit() else None  # convert safely
                df.loc[len(df)] = [player_name, goals, fifa_version]

In [96]:
df

,name,goals,season
0,Mohamed Salah,32,18
1,Harry Kane,30,18
2,Sergio Agüero,21,18
3,Jamie Vardy,20,18
4,Raheem Sterling,18,18
...,...,...,...
244,Joe Lolley,1,18
245,Nacer Chadli,1,18
246,Jan Bednarek,1,18
247,James Morrison,1,18


In [99]:
# Now I can reduce the redundancy with functions and run this through all of the requested seasons